# Cración de dataset con datos de electroencefalografía

In [1]:
import pandas as pd
import sys
import os
import torch
import torchaudio.transforms as T
import numpy as np
import mne
import librosa
import utilit_espectrograms as ue

In [2]:
DATA_DIR        = '/media/martin/Disco2/Rns_Data/RNS_ESPM_datatransfer/Data'
OUTDATA_DIR     = '/media/martin/Disco2/Rns_Data/PITT_PI_DATA/'
OUTMETADATA_DIR = '/media/martin/Disco2/Rns_Data/PITT_PI_DATA/METADATA/'

In [3]:
# crear las carpetas en caso de que no existan

if not os.path.exists(OUTDATA_DIR):
    os.makedirs(OUTDATA_DIR)

if not os.path.exists(OUTMETADATA_DIR):
    os.makedirs(OUTMETADATA_DIR)

In [4]:
# lista con ids de pacientes

RNSIDS = ue.get_subfolders(DATA_DIR)

In [5]:
df = pd.DataFrame(columns=['rns_id', 'data', 'label', 'time'])

ejemplo de un nfile

In [15]:
s      = 1
nepoch = 3
#nfile  = 23

In [16]:
data_files  = ue.get_data_files(DATA_DIR, RNSIDS[s], Verbose=False)
annot_files = ue.get_annot_files(DATA_DIR, RNSIDS[s], Verbose=False)

In [17]:
data_files

['/media/martin/Disco2/Rns_Data/RNS_ESPM_datatransfer/Data/PIT-RNS6989/PIT-RNS6989_PE20190319-1.EDF',
 '/media/martin/Disco2/Rns_Data/RNS_ESPM_datatransfer/Data/PIT-RNS6989/PIT-RNS6989_PE20190423-1.EDF',
 '/media/martin/Disco2/Rns_Data/RNS_ESPM_datatransfer/Data/PIT-RNS6989/PIT-RNS6989_PE20190507-1.EDF',
 '/media/martin/Disco2/Rns_Data/RNS_ESPM_datatransfer/Data/PIT-RNS6989/PIT-RNS6989_PE20190625-1.EDF',
 '/media/martin/Disco2/Rns_Data/RNS_ESPM_datatransfer/Data/PIT-RNS6989/PIT-RNS6989_PE20190708-1.EDF',
 '/media/martin/Disco2/Rns_Data/RNS_ESPM_datatransfer/Data/PIT-RNS6989/PIT-RNS6989_PE20190709-1.EDF',
 '/media/martin/Disco2/Rns_Data/RNS_ESPM_datatransfer/Data/PIT-RNS6989/PIT-RNS6989_PE20190903-1.EDF',
 '/media/martin/Disco2/Rns_Data/RNS_ESPM_datatransfer/Data/PIT-RNS6989/PIT-RNS6989_PE20190903-2.EDF',
 '/media/martin/Disco2/Rns_Data/RNS_ESPM_datatransfer/Data/PIT-RNS6989/PIT-RNS6989_PE20191015-1.EDF',
 '/media/martin/Disco2/Rns_Data/RNS_ESPM_datatransfer/Data/PIT-RNS6989/PIT-RNS6989

In [18]:
RNSIDS[s]

'PIT-RNS6989'

In [19]:
# signal
events  = ue.get_events(annot_files[nepoch])
short_file, time2pad, num_stim_samples= ue.get_padded_and_estim_time(data_files[nepoch], events)

Extracting EDF parameters from /media/martin/Disco2/Rns_Data/RNS_ESPM_datatransfer/Data/PIT-RNS6989/PIT-RNS6989_PE20190625-1.EDF...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


In [20]:
short_file

5

In [21]:
time2pad

[0.196, 0.164, 0.076, 0.088, 0.10400000000000001]

In [22]:
num_stim_samples

np.float64(3.9737024999999995)

In [14]:
label_time.astype(np.float32)

NameError: name 'label_time' is not defined

fin ejemplo de un nfile

In [ ]:
for s in range(len(RNSIDS)):
    
    print('Running subject ' + RNSIDS[s] + ' [s]: ' + str(s))
    data_files  = ue.get_data_files(DATA_DIR, RNSIDS[s], Verbose=False)
    annot_files = ue.get_annot_files(DATA_DIR, RNSIDS[s], Verbose=False)

    for nepoch in range(len(data_files)):

        events                                         = ue.get_events(annot_files[nepoch])
        short_file, time2pad, num_stim_samples, n_stim = ue.get_padded_and_estim_time(data_files[nepoch], events)

        [nt, nc, ns] = np.shape(X)        

        hosp_id, subject_id, PE_id = ue.get_patient_PE(data_files[nepoch], RNSIDS[s])

        for nfile in range(nt):
            file   = X[nfile, :, :]
            signal = torch.from_numpy(file)
            signal = (signal - signal.mean()) / signal.std()
            signal = signal.to(torch.float32)

            # label
            label_time = np.zeros(2)
            label_time[0] = labels[nfile]
            label_time[1] = times[nfile]
            label_time.astype(np.float32)

            data      = {'iEEG': signal, 'label_time': label_time}
            file_name = hosp_id + '_' + subject_id + '_' + PE_id + '_E' + str(nfile)

            df_aux = {  
                        'rns_id': hosp_id + '-' + subject_id, 
                        'data'  : file_name,
                        'label' : labels[nfile],
                        'time'  : times[nfile]
                     }
            
            df = pd.concat([df, pd.DataFrame([df_aux])], ignore_index=True)

            np.save(OUTDATA_DIR + file_name, data)

    df.to_csv(OUTMETADATA_DIR + 'allfiles_metadata.csv', index=False)